In [3]:
# conda env: pyg (Python 3.9.16import copy
import glob
import logging
import os
import re
import subprocess
import sys
import random
from datetime import datetime
from functools import partial

import numpy as np
import torch
from torch import optim

try:
    import wandb
except ImportError:
    wandb = None

try:
    import torch.utils.tensorboard as tensorboard
except ImportError:
    tensorboard = None

try:
    import horovod.torch as hvd
except ImportError:
    hvd = None


In [13]:
#################   Yu's ##########################

#--> from open_clip import create_model_and_transforms, trace_model, get_tokenizer, create_loss
#--> from open_clip_train.data import get_data
from datacat4ml.Scripts.model_dev.rep_open_clip.src.open_clip_train.distributed import is_master, init_distributed_device, broadcast_object
from datacat4ml.Scripts.model_dev.rep_open_clip.src.open_clip_train.logger import setup_logging
from datacat4ml.Scripts.model_dev.rep_open_clip.src.open_clip_train.params import parse_args
#--> from open_clip_train.scheduler import cosine_lr, const_lr, const_lr_cooldown
#--> from open_clip_train.train import train_one_epoch, evaluate
#--> from open_clip_train.file_utils import pt_load, check_exists, start_sync_process, remote_sync

#################   Yu's ##########################

In [5]:
LATEST_CHECKPOINT_NAME = "epoch_latest.pt"

In [ ]:
def random_seed(seed=42, rank=0):
    torch.manual_seed(seed + rank)
    np.random.seed(seed + rank)
    random.seed(seed + rank)

def natural_key(string_):
    """See http://www.codinghorror.com/blog/archives/001018.html"""
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_.lower())]

def get_latest_checkpoint(path: str, remote : bool):
    # as writen, this glob recurses, so can pick up checkpoints across multiple sub-folders
    if remote:
        result = subprocess.run(["aws", "s3", "ls", path + "/"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result)
        if result.returncode == 1:
            return None
        checkpoints = [os.path.join(path, x.split(' ')[-1]) for x in result.stdout.decode().split('\n')[:-1]]
    else:
        checkpoints = glob.glob(path + '**/*.pt', recursive=True)
    if checkpoints:
        checkpoints = sorted(checkpoints, key=natural_key)
        return checkpoints[-1]
    return None

def copy_codebase(args):
    """
    Create a backup of the current codebase (source files) into a specified directory. 
    It is often used in machine learning experiments to ensure reproducibility by saving a snapshot of the code that was used to produce specific results
    """
    from shutil import copytree, ignore_patterns
    new_code_path = os.path.join(args.logs, args.name, "code")
    if os.path.exists(new_code_path):
        print(
            f"Error. Experiment already exists at {new_code_path}. Use --name to specify a new experiment."
        )
        return -1
    print(f"Copying codebase to {new_code_path}")
    current_code_path = os.path.realpath(__file__)
    for _ in range(3):
        current_code_path = os.path.dirname(current_code_path)
    copytree(current_code_path, new_code_path, ignore=ignore_patterns('log', 'logs', 'wandb'))
    print("Done copying code.")
    return 1

# `def main(args):`

In [ ]:
def main(args):
    """
    Yu: Be careful the differnce between `args` and `params` in the original code
    """
    args = parse_args(args)

    if torch.cuda.is_available():
        # This enables tf32 on Ampere GPUs which is only 8% slower than
        # float16 and almost as accurate as float32
        # This was a default in pytorch until 1.12
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
    
    # Fully initialize distributed device environment
    device = init_distributed_device(args)

    # Get the name of the experiments
    if args.name is None:
        # sanitize model name for filesystem / uri use, easier if we don't use / in name as a rule?
        smi_model_name_safe = args.model_name.replace("/", "_")
        date_str = datetime.now().strftime("%Y_%m_%d-%_H%M_%S")
        if args.distributed: #--> Yu's: no argument 'distributed' defined in `params.py``
            # sync data_str from master to all ranks
            date_str = broadcast_object(date_str, device)
        args.name = '-'.join([
            date_str,
            f"smi_model_{smi_model_name_safe}",
            f"lr_{args.lr}",
            f"b_{args.batch_size}", #--> Yu's: be careful, it is 'batch-size' in `params.py`
            f"j_{args.workers}",
            f"p_{args.precision}",
        ])

    resume_latest = args.resume == "latest"
    log_base_path  = os.path.join(args.logs, args.name)
    args.log_path = None #--> Yu's: no argument 'log_path' defined in `params.py`
    if is_master(args, local=args.log_local): #--> Yu's: be careful, it is 'log-local' in `params.py`
        os.makedirs(log_base_path, exist_ok=True)
        log_filename = f'out-{args.rank}' if args.log_local else 'out.log' #--> Yu's: no argument 'rank' defined; and it is 'log-local' in `params.py`
        args.log_path = os.path.join(log_base_path, log_filename)
        if os.path.exists(args.log_path) and not resume_latest:
            print(
                "Error. Experiment already exists. Use --name {} to specify a new experiment."
            )
            return -1   

    # Setup text logger
    args.log_level = logging.DEBUG if args.debug else logging.INFO #--> Yu's: no argument 'log_level' defined in `params.py`
    setup_logging(args.log_path, args.log_level)

    # Setup wandb, tensorboard, checkpoint logging
    args.wandb = 'wandb' in args.report_to or 'all' in args.report_to
    args.tensorboard = 'tensorboard' in args.report_to or 'all' in args.report_to
    args.checkpoint_path = os.path.join(log_base_path, "checkpoints")
    if is_master(args):
        args.tensorboard_path = os.path.join(log_base_path, "tensorboard") if args.tensorboard else ''
        for dirname in [args.tensorboard_path, args.checkpoint_path]:
            if dirname:
                os.makedirs(dirname, exist_ok=True)
    else:
        args.tensorboard_path = ''

    if resume_latest:
        resume_from = None
        checkpoint_path = args.checkpoint_path

        #################   Yu's ##########################
        # Delete the code snippet below if the argument 'remote_sync' is not used
        # If using remote_sync, need to check the remote instead of the local checkpoints folder.
        #if args.remote_sync is not None:
        #    checkpoint_path = os.path.join(args.remote_sync, args.name, "checkpoints")
        #    if args.save_most_recent:
        #        print('Error. Cannot use save-most-recent with remote_sync and resume latest.')
        #        return -1
        #    if args.remote_sync_protocol != 's3':
        #        print('Error. Sync protocol not supported when using resume latest.')
        #        return -1
        #################   Yu's ##########################
        if is_master(args):
            # Checking for existing checkpoint via master rank only. It is possible for
            # different rank processes to see different files if a shared file-system is under
            # stress, however it's very difficult to fully work around such situations.
            if args.save_most_recent:
                # if --save-most-recent flag is set, look for latest at a fixed filename
                resume_from = os.path.join(checkpoint_path, LATEST_CHECKPOINT_NAME)
                if not os.path.exists(resume_from):
                    # If no latest checkpoint has been saved yet, don't try to resume
                    resume_from = None
            else:
                # otherwise, list checkpoint dir contents and pick the newest checkpoint
                resume_from = get_latest_checkpoint(checkpoint_path, remote=args.remote_sync is not None)
            if resume_from:
                logging.info(f'Found latest resume checkpoint at {resume_from}.')
            else:
                logging.info(f'No latest resume checkpoint found in {checkpoint_path}.')
        if args.distributed:
            # sync found checkpoint path to all ranks
            resume_from = broadcast_object(args, resume_from)
        args.resume = resume_from

    if args.copy_codebase:
        copy_codebase(args)

    if args.precision == 'fp16':
        logging.warning(
            'It is recommended to use AMP mixed-precision instead of FP16. '
            'FP16 support needs further verification and tuning, especially for train.')

    if args.horovod:
        logging.info(
            f'Running in horovod mode with multiple processes / nodes. Device: {args.device}.'
            f'Process (global: {args.rank}, local {args.local_rank}), total {args.world_size}.')
    elif args.distributed:
        logging.info(
            f'Running in distributed mode with multiple processes. Device: {args.device}.'
            f'Process (global: {args.rank}, local {args.local_rank}), total {args.world_size}.')
    else:
        logging.info(f'Running with a single process. Device {args.device}.')

    dist_model = None
    args.distill = args.distill_model is not None and args.distill_pretrained is not None
    if args.distill:
        #FIXME: support distillation with grad accum.
        assert args.accum_freq == 1
        #FIXME: support distillation with coca.
        assert 'coca' not in args.model.lower()

    #################   Yu's ##########################
    # the `image` related arguments should be replaced by `smiles` related arguments
    if isinstance(args.force_image_size, (tuple, list)) and len(args.force_image_size) == 1:
        # arg is nargs, single (square) image size list -> int
        args.force_image_size = args.force_image_size[0]
    #################   Yu's ##########################
    random_seed(args.seed, 0)
    model_kwargs = {}
    if args.siglip:
        model_kwargs['init_logit_scale'] = np.log(10)  # different from CLIP
        model_kwargs['init_logit_bias'] = -10
    
    #################   Yu's ##########################
    #--> need to modify the function `create_model_and_transforms` to be with SMILES encoder
    model, preprocess_train, preprocess_val = create_model_and_transforms(
        args.model,
        args.pretrained,
        precision=args.precision,
        device=device,
        jit=args.torchscript,
        force_quick_gelu=args.force_quick_gelu,
        force_custom_text=args.force_custom_text,
        force_patch_dropout=args.force_patch_dropout,
        force_image_size=args.force_image_size,
        image_mean=args.image_mean,
        image_std=args.image_std,
        image_interpolation=args.image_interpolation,
        image_resize_mode=args.image_resize_mode,  # only effective for inference
        aug_cfg=args.aug_cfg,
        pretrained_image=args.pretrained_image,
        output_dict=True,
        cache_dir=args.cache_dir,
        **model_kwargs,
    )
    #################   Yu's ##########################

    if args.distill:
        # FIXME: currently assumes the model you're distilling from has the same tokenizer & transforms.
        dist_model, _, _ = create_model_and_transforms(
            args.distill_model, 
            args.distill_pretrained,
            device=device,
            precision=args.precision,
            output_dict=True,
            cache_dir=args.cache_dir,
        )
    if args.use_bnb_linear is not None:
        print('=> using a layer from bitsandbytes.\n'
              '   this is an experimental feature which requires two extra pip installs\n'
              '   pip install bitsandbytes triton'
              '   please make sure to use triton 2.0.0')
        import bitsandbytes as bnb
        from open_clip.utils import replace_linear
        print(f'=> replacing linear layers with {args.use_bnb_linear}')
        linear_replacement_cls = getattr(bnb.nn.triton_based_modules, args.use_bnb_linear)
        replace_linear(model, linear_replacement_cls)
        model = model.to(device)

    random_seed(args.seed, args.rank)

    if args.trace:
        model = trace_model(model, batch_size=args.batch_size, device=device)

    #################   Yu's ##########################
    #--> need to modify the function `lock_image_tower` to be with SMILES encoder
    #--> if args.lock_image:
    #-->     # lock image tower as per LiT - https://arxiv.org/abs/2111.07991
    #-->     model.lock_image_tower(
    #-->         unlocked_groups=args.lock_image_unlocked_groups,
    #-->         freeze_bn_stats=args.lock_image_freeze_bn_stats)
    #################   Yu's ##########################
    if args.lock_text:
        model.lock_text_tower(
            unlocked_layers=args.lock_text_unlocked_layers,
            freeze_layer_norm=args.lock_text_freeze_layer_norm)
    
    if args.grad_checkpointing:
        model.set_grad_checkpointing()

    if is_master(args):
        logging.info("Model:")
        logging.info(f"{str(model)}")
        logging.info("Params:")
        params_file = os.path.join(args.logs, args.name, "params.txt")
        with open(params_file, "w") as f:
            for name in sorted(vars(args)):
                val = getattr(args, name)
                logging.info(f"  {name}: {val}")
                f.write(f"{name}: {val}\n")

    if args.distributed and not args.horovod:
        if args.use_bn_sync:
            model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
        ddp_args = {}
        if args.ddp_static_graph:
            # this doesn't exist in older PyTorch, arg only added if enabled
            ddp_args['static_graph'] = True
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[device], **ddp_args)
    
        if args.distill:
            dist_model = torch.nn.parallel.DistributedDataParallel(dist_model, device_ids=[device], **ddp_args)
    
    ####
    # ....
    ####

    # initialize datasets
    tokenizer = get_tokenizer(args.model, cache_dir=args.cache_dir)
    data = get_data(
        args,
        (preprocess_train, preprocess_val),
        epoch=start_epoch,
        tokenizer=tokenizer,
    )
    assert len(data), 'At least one train or eval dataset must be specified.'

# Run `main`

- `sys.argv[0]`: The name of the script
- `sys.argv[1:]`: contains the actual arguments passed to the script, i.e. everything after the script name

In [ ]:
main(sys.argv[1:])

usage: ipykernel_launcher.py [-h] [--train-data TRAIN_DATA]
                             [--val-data VAL_DATA]
                             [--train-num-samples TRAIN_NUM_SAMPLES]
                             [--val-num-samples VAL_NUM_SAMPLES]
                             [--dataset-type {webdataset,csv,synthetic,auto}]
                             [--csv-separator CSV_SEPARATOR]
                             [--csv-img-key CSV_IMG_KEY]
                             [--csv-caption-key CSV_CAPTION_KEY]
                             [--imagenet-val IMAGENET_VAL]
                             [--cache-dir CACHE_DIR] [--logs LOGS]
                             [--log-local] [--name NAME] [--workers WORKERS]
                             [--batch-size BATCH_SIZE] [--epochs EPOCHS]
                             [--epochs-cooldown EPOCHS_COOLDOWN] [--lr LR]
                             [--beta1 BETA1] [--beta2 BETA2] [--eps EPS]
                             [--wd WD] [--momentum MOMENTUM] [--warmup

SystemExit: 2

/storage/homefs/yc24j783/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
